In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from keras.models import model_from_json
from skimage.transform import resize

Using TensorFlow backend.


In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename):   
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)
    
    # Check Patient Position
    if ds.PatientPosition not in ['AP', 'PA']:
        print("Error in file {} - Patient Position is invalid (Expected: AP/PA - ACTUAL: {})".format(filename, ds.PatientPosition))
        return None
    
    # Check Image Type
    if ds.Modality != 'DX':
        print("Error in file {} - Image Type is invalid. (Expected: DX - ACTUAL: {})".format(filename, ds.Modality))
        return None
    
    # Check Body Part
    if ds.BodyPartExamined != 'CHEST':
        print("Error in file {} - Body Part Examined is invalid. (Expected: CHEST - ACTUAL: {})".format(filename, ds.BodyPartExamined))
        return None
    
    img = ds.pixel_array
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    proc_img = (img/255.0-img_mean)/img_std
    proc_img = resize(proc_img, img_size)
    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    with open(model_path, 'r') as json_file:
        model_json = json_file.read()
        model = model_from_json(model_json)
        model.load_weights(weight_path)    
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    pred_Y = model.predict(img)   
    if pred_Y > thresh:
        return 'Pneumonia'
    else:
        return 'Not pneumonia'
    
    return prediction 

In [3]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json'
weight_path = 'xray_class_my_model.best.hdf5'

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = 0.492
img_std = 0.250

my_model = load_model(model_path, weight_path)
thresh = 0.72

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Load file test1.dcm ...
Not pneumonia
Load file test2.dcm ...
Not pneumonia
Load file test3.dcm ...
Not pneumonia
Load file test4.dcm ...
Error in file test4.dcm - Body Part Examined is invalid. (Expected: CHEST - ACTUAL: RIBCAGE)
Load file test5.dcm ...
Error in file test5.dcm - Image Type is invalid. (Expected: DX - ACTUAL: CT)
Load file test6.dcm ...
Error in file test6.dcm - Patient Position is invalid (Expected: AP/PA - ACTUAL: XX)
